In [1]:
from HSmodule import *
from source.SimHashDetection import SimHashDetection
from source.minHashDetection import MinHashDetection
from source.BloomDetection import BloomDetection
from source.FaissSearch import FaissSearch
from source.Preprocessor import *
import numpy as np
import pandas as pd

e:\DSA\Extended_assign\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 1. Đọc và chọn cột (làm lại từ đầu cho chắc)
df_large = pd.read_csv('dataset/dataset_small.csv')
df_large = df_large[['question1', 'question2']]

# 2. Nối 2 cột thành một Series
s = pd.concat([df_large['question1'], df_large['question2']])

# 3. DÙNG .dropna() ĐỂ XÓA TẤT CẢ GIÁ TRỊ RỖNG (NaN)
s_clean = s.dropna()

# 4. Bây giờ mới chuyển thành list
listText = s_clean.tolist()

# Bây giờ listText của bạn sẽ sạch và chỉ chứa chuỗi

In [ ]:
model1 = BloomDetection()
model2 = MinHashDetection()
model3 = SimHashDetection()
listModel = [model1, model2, model3]
model_names = ['BloomDetection', 'MinHashDetection', 'SimHashDetection']

In [ ]:
import sys
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from memory_profiler import memory_usage

# ==============================================================================
# 1. KHAI BÁO MODEL VÀ DỮ LIỆU (TỪ BÊN NGOÀI)
#
# !!! QUAN TRỌNG !!!
# Giả định rằng bạn đã import hoặc định nghĩa các class:
# - BloomDetection
# - MinHashDetection
# - SimHashDetection
#
# Và đã chuẩn bị biến `listText` (danh sách 10,000 văn bản)
#
# Ví dụ (bạn không cần code này nếu đã import):
# from your_models import BloomDetection, MinHashDetection, SimHashDetection
# from your_data import listText
#
# ==============================================================================


# ==============================================================================
# 2. HÀM HELPER ĐỂ TÍNH KÍCH THƯỚC INPUT (MB)
# (Giữ nguyên)
# ==============================================================================

def get_list_size_mb(text_list: list) -> float:
    """
    Tính toán tổng kích thước bộ nhớ (tính bằng MB) của một danh sách các chuỗi.
    Bao gồm kích thước của container (list) và kích thước của mỗi chuỗi bên trong.
    """
    total_bytes = sys.getsizeof(text_list)  # Kích thước của chính cái list (các con trỏ)
    for s in text_list:
        total_bytes += sys.getsizeof(s)  # Cộng thêm kích thước của mỗi chuỗi
    return total_bytes / (1024 * 1024)

# ==============================================================================
# 3. CHUẨN BỊ MODEL VÀ ĐO LƯỜNG BỘ NHỚ KHỞI TẠO
# ==============================================================================

print("Đo lường bộ nhớ khởi tạo của Model (Init Memory)...")

# Hàm bọc (wrapper) để đo lường bộ nhớ khi khởi tạo
def init_model(model_class):
    return model_class()

# Đo lường mức tăng bộ nhớ khi khởi tạo từng model
baseline_before_all = memory_usage()[0]

mem_usage_bloom = memory_usage((init_model, (BloomDetection,)), max_usage=True)
init_mem_bloom = mem_usage_bloom - baseline_before_all

mem_usage_minhash = memory_usage((init_model, (MinHashDetection,)), max_usage=True)
init_mem_minhash = mem_usage_minhash - baseline_before_all

mem_usage_simhash = memory_usage((init_model, (SimHashDetection,)), max_usage=True)
init_mem_simhash = mem_usage_simhash - baseline_before_all

# Lưu bộ nhớ khởi tạo (tính bằng MiB)
model_init_memory = {
    'BloomDetection': init_mem_bloom,
    'MinHashDetection': init_mem_minhash,
    'SimHashDetection': init_mem_simhash,
}

print(f"  - BloomDetection Init: {init_mem_bloom:.2f} MiB")
print(f"  - MinHashDetection Init: {init_mem_minhash:.2f} MiB")
print(f"  - SimHashDetection Init: {init_mem_simhash:.2f} MiB")

# Khởi tạo các model để sử dụng
print("\nKhởi tạo các model để chạy thử nghiệm...")
model1 = BloomDetection()
model2 = MinHashDetection()
model3 = SimHashDetection()
listModel = [model1, model2, model3]
model_names = ['BloomDetection', 'MinHashDetection', 'SimHashDetection']

# Xác nhận dữ liệu listText
print(f"Sử dụng {len(listText)} văn bản có sẵn.")
print(f"Ước tính tổng kích thước dữ liệu: {get_list_size_mb(listText):.2f} MB")


# ==============================================================================
# 4. CHẠY THỬ NGHIỆM VÀ ĐO LƯỜNG BỘ NHỚ
# ==============================================================================

results_list = []
num_steps = 20
amounts_to_test = np.linspace(1000, 10000, num_steps, dtype=int)

print(f"\nBắt đầu đo lường trên {num_steps} bước (từ 1000 đến 10000)...")

for amount in amounts_to_test:
    print(f"\n--- Đang xử lý {amount} văn bản ---")
    sub_list = listText[:amount]
    
    # 1. Tính toán trục X (Kích thước Input thực tế)
    input_mb = get_list_size_mb(sub_list)
    
    for model, name in zip(listModel, model_names):
        print(f"  Đo lường Model: {name}...")
        
        try:
            # 2. Đo lường trục Y (Sử dụng Bộ nhớ)
            # Chúng ta gọi hàm memory_usage để chạy hàm model.detect(sub_list)
            # Nó trả về một danh sách các mốc bộ nhớ (tính bằng MiB) được ghi lại
            mem_readings_tuple = memory_usage(
                (model.detect, (sub_list,)), # (hàm, (args,))
                interval=0.01, # Tần suất lấy mẫu (giây)
                timeout=120,    # Thời gian tối đa (giây)
                retval=True,    # Trả về cả kết quả của hàm (nếu cần)
                max_usage=False # Quan trọng: Trả về list, không chỉ max
            )
            
            mem_readings = mem_readings_tuple[0] # Lấy danh sách mốc bộ nhớ
            
            if not mem_readings:
                print(f"    LỖI: Không thể đo lường bộ nhớ cho {name}.")
                continue
                
            # Tính toán mức tăng bộ nhớ TỐI ĐA (chỉ của hàm detect)
            # (Peak memory - Baseline memory at start of call)
            peak_mem = max(mem_readings)
            baseline_mem = mem_readings[0]
            mem_increment = peak_mem - baseline_mem
            
            # TÍNH TOÁN TỔNG BỘ NHỚ (THEO YÊU CẦU MỚI)
            # Tổng = Bộ nhớ Khởi tạo + Bộ nhớ Tăng thêm khi chạy
            init_memory = model_init_memory[name]
            total_memory_used = init_memory + mem_increment
            
            print(f"Name: {name}, Init Mem: {init_memory:.2f} MiB, Input: {input_mb:.2f} MB, Mem Incr: {mem_increment:.2f} MiB, Total: {total_memory_used:.2f} MiB")

            # 3. Lưu kết quả
            results_list.append({
                'Model': name,
                'Amount': amount,        # Số lượng (để tham khảo)
                'Input (MB)': input_mb,  # Trục X
                'Memory (MiB)': total_memory_used # Trục Y (ĐÃ CẬP NHẬT)
            })
            

            
            print(f"    -> Input: {input_mb:.2f} MB, "
                  f"Init: {init_memory:.2f} MiB, "
                  f"Detect Incr: {mem_increment:.2f} MiB, "
                  f"Tổng: {total_memory_used:.2f} MiB")

        except Exception as e:
            print(f"    LỖI khi chạy {name}: {e}")



In [ ]:
# ==============================================================================
# 5. VẼ BIỂU ĐỒ ĐƯỜNG
# ==============================================================================

print("\nTính toán hoàn tất. Đang vẽ biểu đồ...")
df_results = pd.DataFrame(results_list)

if df_results.empty:
    print("Không có dữ liệu để vẽ. Đã xảy ra lỗi trong quá trình đo lường.")
else:
    print("\n--- Kết quả đo lường (5 dòng đầu) ---")
    print(df_results.head())

    # Khởi tạo figure
    fig, ax = plt.subplots(figsize=(12, 7))
    
    # Cập nhật tiêu đề và trục Y
    ax.set_title("So sánh Tổng Sử dụng Bộ nhớ của các Model (Khởi tạo + Xử lý)", fontsize=16)
    # Trục X là Kích thước Input (MB)
    ax.set_xlabel("Kích thước Input (MB)", fontsize=12)
    # Trục Y là TỔNG bộ nhớ (MiB)
    ax.set_ylabel("Tổng Bộ nhớ Sử dụng (Init + Peak) (MiB)", fontsize=12)
    ax.grid(True, linestyle='--', alpha=0.6)

    # Vẽ từng đường cho mỗi model
    for name in model_names:
        model_df = df_results[df_results['Model'] == name]
        if not model_df.empty:
            # Sắp xếp theo Input (MB) để đảm bảo đường vẽ đúng
            model_df = model_df.sort_values(by='Input (MB)')
            ax.plot(
                model_df['Input (MB)'], 
                model_df['Memory (MiB)'], 
                marker='o', 
                markersize=5, 
                label=name
            )

    ax.legend(title="Model")
    plt.tight_layout()
    
    # Hiển thị biểu đồ
    print("\nHiển thị biểu đồ. Đóng cửa sổ biểu đồ để kết thúc.")
    plt.show()

print("Hoàn tất.")